In [4]:
scientific_facts = [
    "Isaac Newton discovered gravity.",
    "Gravity is a fundamental force in physics.",
    "Albert Einstein developed the theory of relativity.",
    "The theory of relativity explains gravity at cosmic scales.",
    "Marie Curie discovered radium.",
    "Radium is a radioactive element."
]

In [5]:
import ollama 

def extract_triples(text):
    model_name = 'llama3.1:8b'
    # model_name = 'deepseek-r1:8b'

    print(f'LLM model used: {model_name}')
    # prompt = f"Answer the following question based on the article:\n\n{context}\n\nQuestion: {query}\nAnswer:"
    prompt = prompt = (
            "Extract the subject, relation, and object from the following sentence as a triple:\n"
            f"Sentence: {text}\n"
            "Triple:"
        )
    messages=[
        {
            'role' : 'user',
            'content' : prompt,
            "parameters": {"max_new_tokens":1000, "temperature":0.1},
        },
    ]
    response = ollama.chat(
        model=model_name, 
        messages=messages,
    )

    OllamaResponse = response['message']['content'] 
    triple = OllamaResponse.strip()
    return tuple(map(str.strip, triple.split(',')))

In [12]:
def extract_triples_openai(text):
    prompt = (
        "Extract the subject, relation, and object from the following sentence as a triple:\n"
        f"Sentence: {text}\n"
        "Triple:"
    )
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=60,
        temperature=0
    )
    triple = response.choices[0].text.strip()
    # Example output: "Isaac Newton, discovered, gravity"
    return tuple(map(str.strip, triple.split(',')))

In [8]:
import networkx as nx

kg = nx.MultiDiGraph()  # Use MultiDiGraph to allow multiple edges between nodes

for fact in scientific_facts:
    try:
        subj, rel, obj = extract_triples(fact)
        kg.add_edge(subj, obj, label=rel)
    except Exception as e:
        print(f"Could not extract triple from: {fact} -- {e}")

print("Nodes:", list(kg.nodes))
print("Edges:")
for u, v, data in kg.edges(data=True):
    print((u, v, data))

LLM model used: llama3.1:8b
Could not extract triple from: Isaac Newton discovered gravity. -- not enough values to unpack (expected 3, got 1)
LLM model used: llama3.1:8b
Could not extract triple from: Gravity is a fundamental force in physics. -- not enough values to unpack (expected 3, got 1)
LLM model used: llama3.1:8b
Could not extract triple from: Albert Einstein developed the theory of relativity. -- not enough values to unpack (expected 3, got 1)
LLM model used: llama3.1:8b
Could not extract triple from: The theory of relativity explains gravity at cosmic scales. -- not enough values to unpack (expected 3, got 1)
LLM model used: llama3.1:8b
Could not extract triple from: Marie Curie discovered radium. -- not enough values to unpack (expected 3, got 1)
LLM model used: llama3.1:8b
Could not extract triple from: Radium is a radioactive element. -- not enough values to unpack (expected 3, got 1)
Nodes: []
Edges:


In [ ]:
import matplotlib.pyplot as plt

G = nx.petersen_graph()

subax1 = plt.subplot(121)
nx.draw(G, with_labels=True, font_weight='bold')
subax2 = plt.subplot(122)
nx.draw_shell(G, kg, with_labels=True, font_weight='bold')